In [2]:
import os

import numpy as np
from tqdm import tqdm
from scipy.stats import truncnorm
import sys

sys.path.append('../')
from Agents.A2CAgent import Agent
from Boards.Speed_leedo_2p_v3 import FullBoard
from Intelligent_bot.intelligent_bot import Bot
import gc

env = FullBoard()
agent0 = Agent(n_actions=env.action_size(), input_dim=env.state_size(), alpha=1e-8, max_val=env.max_val())
bot0 = Bot()

wins = []

# output_dir_a = '../trainers_vs_intbot/model_output_vs_intbot/A2C_v6/0001/actor/weights_11300.hdf5'
# output_dir_c = '../trainers_vs_intbot/model_output_vs_intbot/A2C_v6/0001/critic/weights_11300.hdf5'

output_dir_a = '../model_output/A2C_v5/0001/actor/weights_16000.hdf5'
output_dir_c = '../model_output/A2C_v5/0001/critic/weights_16000.hdf5'

if not os.path.exists(output_dir_a):
    os.makedirs(output_dir_a)
if not os.path.exists(output_dir_c):
    os.makedirs(output_dir_c)

agent0.load_model(output_dir_a, output_dir_c)

for ep in tqdm(range(0, 1000), ascii=True, unit="e"):
    s, _, game_over, player_turn = env.reset()
    episode_reward = [0.0, 0.0]

    time = 0
    while time < 360 or player_turn == 0:

        time += truncnorm.rvs(-2.5, 2.5, loc=2.5, scale=1)
        player_turn_temp = env.get_player_turn()
        env.roll_dice()[0]
        player_turn = env.get_player_turn()

        if player_turn == player_turn_temp:

            action_list = env.get_next_states(player_turn)

            if action_list:
                if player_turn == 1:
                    s_t = env.convert_state(player_turn)
                    action = agent0.act(s_t, action_list)
                else:
                    s_t = env.return_state()
                    action = bot0.act(state=s_t, p=player_turn)

                time += truncnorm.rvs(-2.5, 2.5, loc=2.5, scale=1)
                s_, reward, game_over, player_turn_temp = env.make_step(action)

                episode_reward[player_turn] += reward[player_turn]
       
    if episode_reward[1] > episode_reward[0]:
        wins.append(1)
    else:
        wins.append(0)

  0%|          | 0/1000 [00:00<?, ?e/s]/nfs_storage/fs-mnt6/barathmohanU/ludo_bot/LudoRL/Intelligent_bot/../Intelligent_bot/intelligent_bot.py:140: RuntimeWarning: divide by zero encountered in divide
  self.pawn_priority[i] = self.pawn_priority[i] * 0.1 / self.pawn_score[i]
100%|##########| 1000/1000 [08:47<00:00,  1.90e/s]


In [3]:
np.sum(wins)

0